<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLGymGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI]((https://gym.openai.com)

Voir la page d'[introduction à Gym](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLIntroGym.ipynb).

### Installation de gym

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [1]:
!pip install gym
!pip install Box2D


In [ ]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt
#On MAc : 
#import matplotlib
#matplotlib.use("MacOSX")

#### <font color=red>Sous colab</font>
*Sous colab, l'affichage ne sera pas dynamique.. Il faudra passer soit par un rendu sous forme de plot (matplotlib), soit passer par une vidéo à enregistrer...*

***Solution pour affichage statique***<br>
*Il vous faut importer, seulement si vous utilisez colab, les packages suivants :*

In [2]:
!sudo apt-get update
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graphics-driver

In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)


In [1]:
#GYM
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [2]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt


---
## Charger l' environnement
On utilise ici l'environnement `LunarLander-v2`.


In [7]:
env = gym.make('LunarLander-v2') 

#### <font color=red><b>Sous colab</b></font>

In [3]:
env = wrap_env(gym.make("LunarLander-v2"))

---
### Les valeurs des caractéristiques
Pour `LunarLander-v2`, vous trouverez l'explication des valeurs [ici](https://gym.openai.com/envs/LunarLander-v2/).

Il s'agit de faire atterrir une sonde sur une planète.

L'espace des actions est une valeur discrète dans l'intervalle [0,3] : 
  - 0 : Ne rien faire
  - 1 : allumage du moteur gauche
  - 2 : allumage du moteur central
  - 3 : allumage du moteur droit

Les observations sont au nombre de 8 : 
  - obs[0] : coordonnée horizontale
  - obs[1] : coordonnée verticale
  - obs[2] : vitesse horizontale
  - obs[3] : vitesse verticale
  - obs[4] : angle de la sonde
  - obs[5] : vitesse angulaire (de rotation)
  - obs[6] : 1 si le pied gauche de la sonde est au contact du sol, 0 sinon
  - obs[7] : 1 si le pied droit de la sonde est au contact du sol, 0 sinon

Le but est donc de faire atterrir la sonde dans une position donnée (0,0) avec une vitesse suffisamment faible pour éviter le crash.


In [4]:
print(env.spec)
print("nombre et type d'actions = ", env.action_space) 
print("nombre et type d'observations = ", env.observation_space) 
print("valeurs de  observations les plus basses = ", env.observation_space.low) 
print("valeurs de observations les plus elevées = ", env.observation_space.high) 

EnvSpec(LunarLander-v2)
nombre et type d'actions =  Discrete(4)
nombre et type d'observations =  Box(-inf, inf, (8,), float32)
valeurs de  observations les plus basses =  [-inf -inf -inf -inf -inf -inf -inf -inf]
valeurs de observations les plus elevées =  [inf inf inf inf inf inf inf inf]


### (Re)Initialiser l'environnement
`env.reset()` initialise les variables d'environnement. L'exécution est obligatoire après avoir chargé l'environnement. Ici ce sont les coordonnées, l'angle et les vitesses.

C'est un environnement non déterministe par défaut, `env.reset()` retourne des valeurs légèrement différentes à chaque exécution : 

In [29]:
env.reset()

array([-0.00364351,  1.4210072 , -0.36906666,  0.4482938 ,  0.00422874,
        0.08359913,  0.        ,  0.        ], dtype=float32)

L'algorithme génétique se prète mal aux environnement non stables; aussi nous utiliserons la fonction suivante pour fixer les vitesses et angles initiaux : 

In [5]:
def reset():
    env.reset();
    env.lander.angle=0
    env.lander.angularVelocity = 0
    env.lander.ApplyForceToCenter( (0,0,), False)
    env.lander.linearVelocity = b2.b2Vec2()

---
### Les actions
- Dans `LunarLander-v2` : [-1.0] fait reculer le véhicule d'une force 1, [0] le met au point mort, [1.0] le fait avancer d'une force 1

- ***Exécuter une action :***
  - *exécution* : `observation, reward,done,info = env.step(action)`
    - `observation`: ici les 8 éléments d'observation
    - `reward`: utilité de l'état (récompense 100 si bien posé, -100 si crash)
    - `done` : booléen (True si but (position) atteinte)
    - `info` : éventuelle information retournée par l'outil

**Exemples d'actions**

In [15]:
def test_action(no: int, nb: int = 1):
    """effectue nb fois l'action no dans l'environnement chargé"""
    done:bool = False
    i:int = 0
    while i<nb and not done:
        i += 1
        env.render()
        obs, reward, done, _ = env.step(no)
        print(reward, done)
    return done

def test():
  done = test_action(2, 40)
  if (not done): done = test_action(0, 40)
  if (not done): done = test_action(1, 40)
  if (not done): done = test_action(3, 40)

In [16]:
env = wrap_env(gym.make("LunarLander-v2"))
reset()
test()

36.53142998979878 False
-4.539322218051967 False
-1.492186184289426 False
-4.713936032741077 False
-4.023913745318214 False
-4.031389537123277 False
-4.036947491524518 False
-2.747512772697621 False
-1.9133675593171688 False
-3.6197930762454176 False
-3.5973139500932403 False
-4.080878650723281 False
-4.655643326636226 False
-4.747916535490492 False
-1.4227444263750215 False
-4.82456359436249 False
-5.126926710872897 False
-4.2788684933130865 False
-5.905320216879875 False
-2.2648723431561733 False
-4.340973023891979 False
-4.798524406387815 False
-4.470228855443298 False
-4.6784003631662925 False
-3.0925631047869215 False
-2.0811297808597713 False
-5.719191398295908 False
-5.629898969376273 False
-2.547599267370674 False
-3.8650637440781734 False
-6.119310982893967 False
-5.666954891177499 False
-6.803080889349633 False
-2.3956051882509426 False
-5.004953156176373 False
-2.6366108418667293 False
-3.2053035651652406 False
-6.795889766821278 False
-6.040486767473214 False
-3.70940746894



**Sous Colab**


In [17]:
env.close()
show_video()

ATTENTION, sous colab, bie. répéter la ligne
`env = wrap_env(gym.make("LunarLander-v2"))' avant le reset et la création de vidéos


---

# Test d'algorithme génétique
## Test de programme évolutionnaire pour atteindre l'objectif
- Utiliser l'environnement `LunarLander-v2`, car il est plus rapide 
- Programmez (en python) un algo génétique avec initialement les éléments suivants : 
  - sequence : tableau de valeurs entières 0, 1, 2 ou 3.
  - 1 point de croisement (au centre)
    - *(pour commencer, on pourrait proposer 2 points de croisement (au 1/3, et 2/3))*
  - taille sequence : taille_seq = 150
  - taille population initiale : taille_pop = 500
  - taux de gènes mutants dans une séquence : taux_mut_seq = 0.1
  - taux de séquences mutantes dans la population : taux_mut_pop = 0.2
  - nombre max de cycles de reproductions : nb_cycles = 3000


- Pour tester une séquence, il suffit de balayer le tableau et de lancer l'action correspondante..
- Les 4 meilleures séquences (11,22,33,44) se reproduisent entre elles, donnant donc 12 fils : 12, 13, 14, 21, 23, 24, 31, 32, 34, 41, 42, 43.
- Idéalement, un filtre doit permettre de supprimer les séquences identiques qui pourraient venir des croisements et des mutations.

- Tester votre algo sur le nb de générations et afficher le rendu toutes les 100 générations et bien sûr le rendu de la meilleure séquence finale.
  - (uniquement la position finale *colab*)
  - (sous forme d'animation sous autre outil)


- ***NB:*** 
  - Au plus simple il est possible d'utiliser une population (liste) de couples [sequence, utilite] où l'utilité serait la distance en x.

---
### V2 : Arrivée avec angle 
- Maintenant la sonde arrive avec un angle donné (45 par exemple).
Vérifiez que votre algorithme trouve bien la solution...

In [ ]:
def reset2(angle):
    env.reset();
    env.lander.angle = angle
    env.lander.angularVelocity = 0
    env.lander.ApplyForceToCenter( (
            0,0,
            ), False)
    env.lander.linearVelocity = b2.b2Vec2()

In [ ]:
reset2()
#application de l'algo...